In [1]:
## Ran for eveything now with excluding the gauges that we don't want

# Create a set of dimensionless profiles
Read in all of the events, for all durations, for all gauges, for all ensemble members.  
Convert them to dimensionless profiles, with 12 values between 0 and 1.  
Each value is a dimensionless, cumulative rainfall value (cumulative rainfall at this timestep, normalised by the total event rainfall):
- 0 means no rainfall has occurred, and 
- 1 means the total event rainfall has been reached.  

If there are less than 12 values, then these are filled in with interpolation

In [277]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import pandas as pd
import re
from datetime import datetime

from Create_Profiles_Functions import *

def find_part_with_most_rain(array, n, plot=False, ax= False):
    # Compute differences
    # Split the array into 5 equal parts
    splits = np.array_split(array, n)
    
    max_array_rainfall = 0
    max_array_num = None
    
    total_precipitations = []  # To store total precipitation for each split
    split_ranges = []  # To store start and end indices for each split
    
    # Calculate total precipitation for each split
    split_start = 0
    for split in splits:
        total_precipitation = split.sum()
        total_precipitations.append(total_precipitation)
        split_end = split_start + len(split)
        split_ranges.append((split_start, split_end))
        if total_precipitation > max_array_rainfall:
            max_array_num = len(total_precipitations)
            max_array_rainfall = total_precipitation
        split_start = split_end
    
    colors = ['lightblue'] * n  # Default color for all splits
    highlight_color = 'yellow'  # Color for the split with the most rainfall
    
    if plot:
        # Plot the array
        ax.plot(range(1, len(array) + 1), array, label='Precipitation', marker='o')
        
        # Add vertical lines and shading for each split segment
        for i, (start_index, end_index) in enumerate(split_ranges):
            color = highlight_color if (i + 1) == max_array_num else colors[i]
            
            # Add vertical lines at the start and end of each split
            ax.axvline(x=start_index + 1, color=color, linestyle='--', label=f'Split {i+1} Start' if i == 0 or (i + 1) == max_array_num else "")
            ax.axvline(x=end_index, color=color, linestyle='--', label=f'Split {i+1} End' if i == 0 or (i + 1) == max_array_num else "")
            
            # Shade the region for the split
            ax.fill_between(range(start_index + 1, end_index + 1), array[start_index:end_index], color=color, alpha=0.3)
            
            # Add the total precipitation value behind the shading
            ax.text((start_index + end_index) / 2+0.5, max(array) * 0.05,  # Adjust y-position if needed
                    f'{total_precipitations[i]:.2f}',
                    ha='center', va='center', fontsize=10, color='black', weight='bold', zorder=1)
        
        ax.set_title(f'Precipitation Values with Splits Marked. Max at {max_array_num}')
        ax.set_xlabel('Time')
        ax.set_ylabel('Precipitation')

    return max_array_num


In [2]:
gauge_nums = range(0,1294)

# NIMROD - 30 mins
### Make profiles

In [43]:
for nimrod_option in ["NIMROD_2.2km_filtered_100"]:

    # New list to store just the files that are 'good' e.g. 
    dict_of_things_im_storing ={}
    dict_of_things_im_storing['good_raw_events'] = []
    dict_of_things_im_storing['good_raw_events_seasons'] = []
    dict_of_things_im_storing['good_trimmed_events'] = []
    dict_of_things_im_storing['good_trimmed_events_durations'] = []
    dict_of_things_im_storing['good_normalised_events'] = []
    dict_of_things_im_storing['good_normalised_interpolated_events'] = []
    dict_of_things_im_storing['good_normalised_interpolated_events_max_quintiles'] = []

    for gauge_num in range(0,1293):
        print(f"gauge num {gauge_num}")
        if gauge_num not in [433, 444, 827, 888]:
            counter = 0
            # Create a list of all the event CSVs
            files = [f for f in os.listdir(f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_30mins/{nimrod_option}/{gauge_num}/WholeYear") if f.endswith('.csv')]
            files = np.sort(files)
                
            # Go through each event
            for file in files:
                # Construct the path to the file
                fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_30mins/{nimrod_option}/{gauge_num}/WholeYear/{file}"
                if '2080' in fp:
                    pass
                    # print(f"{fp} 2080 so not using")
                else:
                    # print(f"{fp}not 2080 so using")
                    # Read in event
                    this_event = read_event(gauge_num, fp)
                    # Trim leading and trailing zeroes
                    trimmed_event = remove_leading_and_trailing_zeroes(this_event)

                    # Check trimmed event for 'problems', if there are none, return a dataframe, if there are, return None
                    real_trimmed_event, problem_events = remove_events_with_problems(trimmed_event, verbose=False)

                    # If we still have a dataframe, then add it to list of good, unprocessed events
                    if real_trimmed_event is not None:

                        # Create normalised version of event
                        normalized_time, normalized_rainfall = create_normalised_event(real_trimmed_event)
                        # Create interpolated, normalised version of event (with 12 data points)
                        normalized_interpolated_rainfall = interpolate_and_bin(normalized_time, normalized_rainfall)
                        # Find the fifth OF THE INTERPOLATED, NORMALISED EVENT with the maximum amount of rainfall
                        max_quintile = find_part_with_most_rain(normalized_interpolated_rainfall, 5)
                        duration = len(real_trimmed_event)/2
                        season = get_season(trimmed_event['times'][0])
                                            
                    else:
                        print(f"{file}event not real")
                        normalized_rainfall = None
                        normalized_interpolated_rainfall = None
                        max_quintile = None
                        duration = None
                        season = None

                    # Add to list of events with no problems
                    dict_of_things_im_storing['good_raw_events'].append(this_event)
                    dict_of_things_im_storing['good_raw_events_seasons'].append(season)
                    dict_of_things_im_storing['good_trimmed_events'].append(real_trimmed_event)
                    dict_of_things_im_storing['good_trimmed_events_durations'].append(duration)
                    dict_of_things_im_storing['good_normalised_events'].append(normalized_rainfall)
                    dict_of_things_im_storing['good_normalised_interpolated_events'].append(normalized_interpolated_rainfall)
                    dict_of_things_im_storing['good_normalised_interpolated_events_max_quintiles'].append(max_quintile)            
            

    for file_name,stored_list in dict_of_things_im_storing.items():
        print(file_name)
        cache_filepath = f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/NIMROD_30mins/WholeYear/{nimrod_option}_{file_name}.pkl"
        with open(cache_filepath, 'wb') as f:
            pickle.dump(stored_list, f)                    

gauge num 0
0.5hrs_2018_v2_part0.csvevent not real
gauge num 1
0.5hrs_2007_v2_part0.csvevent not real
gauge num 2
24hrs_2010_v2_part1.csvevent not real
gauge num 3
gauge num 4
gauge num 5
gauge num 6
gauge num 7
0.5hrs_2010_v2_part0.csvevent not real
gauge num 8
gauge num 9
24hrs_2018_v2_part1.csvevent not real
gauge num 10
0.5hrs_2014_v2_part0.csvevent not real
gauge num 11
0.5hrs_2013_v2_part0.csvevent not real
gauge num 12
gauge num 13
0.5hrs_2008_v2_part0.csvevent not real
0.5hrs_2014_v2_part0.csvevent not real
1hrs_2018_v2_part0.csvevent not real
gauge num 14
0.5hrs_2018_v2_part0.csvevent not real
gauge num 15
gauge num 16
1hrs_2017_v2_part0.csvevent not real
gauge num 17
0.5hrs_2018_v2_part0.csvevent not real
gauge num 18
gauge num 19
24hrs_2013_v2_part0.csvevent not real
gauge num 20
gauge num 21
0.5hrs_2014_v2_part0.csvevent not real
gauge num 22
gauge num 23
gauge num 24
gauge num 25
0.5hrs_2007_v2_part0.csvevent not real
gauge num 26
gauge num 27
gauge num 28
gauge num 29
gau

gauge num 146
gauge num 147
0.5hrs_2017_v2_part0.csvevent not real
gauge num 148
0.5hrs_2011_v2_part0.csvevent not real
0.5hrs_2014_v2_part0.csvevent not real
gauge num 149
0.5hrs_2019_v2_part0.csvevent not real
0.5hrs_2020_v2_part0.csvevent not real
gauge num 150
0.5hrs_2019_v2_part0.csvevent not real
1hrs_2019_v2_part0.csvevent not real
gauge num 151
gauge num 152
0.5hrs_2007_v2_part0.csvevent not real
gauge num 153
gauge num 154
0.5hrs_2012_v2_part0.csvevent not real
gauge num 155
0.5hrs_2011_v2_part0.csvevent not real
gauge num 156
gauge num 157
0.5hrs_2019_v2_part0.csvevent not real
1hrs_2019_v2_part0.csvevent not real
gauge num 158
gauge num 159
0.5hrs_2018_v2_part0.csvevent not real
gauge num 160
0.5hrs_2009_v2_part0.csvevent not real
gauge num 161
gauge num 162
0.5hrs_2009_v2_part0.csvevent not real
0.5hrs_2019_v2_part0.csvevent not real
gauge num 163
0.5hrs_2007_v2_part0.csvevent not real
12hrs_2006_v2_part0.csvevent not real
gauge num 164
gauge num 165
gauge num 166
0.5hrs_20

1hrs_2018_v2_part0.csvevent not real
gauge num 305
gauge num 306
0.5hrs_2007_v2_part0.csvevent not real
0.5hrs_2011_v2_part0.csvevent not real
0.5hrs_2013_v2_part0.csvevent not real
1hrs_2011_v2_part0.csvevent not real
gauge num 307
0.5hrs_2009_v2_part0.csvevent not real
gauge num 308
0.5hrs_2015_v2_part0.csvevent not real
gauge num 309
0.5hrs_2013_v2_part0.csvevent not real
gauge num 310
gauge num 311
gauge num 312
gauge num 313
0.5hrs_2011_v2_part0.csvevent not real
gauge num 314
gauge num 315
gauge num 316
gauge num 317
gauge num 318
gauge num 319
0.5hrs_2010_v2_part0.csvevent not real
0.5hrs_2015_v2_part0.csvevent not real
gauge num 320
0.5hrs_2007_v2_part0.csvevent not real
gauge num 321
0.5hrs_2014_v2_part0.csvevent not real
gauge num 322
gauge num 323
0.5hrs_2014_v2_part0.csvevent not real
gauge num 324
0.5hrs_2014_v2_part0.csvevent not real
1hrs_2014_v2_part0.csvevent not real
gauge num 325
0.5hrs_2014_v2_part0.csvevent not real
gauge num 326
24hrs_2007_v2_part1.csvevent not re

0.5hrs_2020_v2_part0.csvevent not real
gauge num 482
gauge num 483
0.5hrs_2018_v2_part0.csvevent not real
gauge num 484
0.5hrs_2012_v2_part0.csvevent not real
0.5hrs_2013_v2_part0.csvevent not real
gauge num 485
gauge num 486
0.5hrs_2006_v2_part0.csvevent not real
gauge num 487
0.5hrs_2008_v2_part0.csvevent not real
1hrs_2008_v2_part0.csvevent not real
gauge num 488
gauge num 489
0.5hrs_2013_v2_part0.csvevent not real
gauge num 490
0.5hrs_2011_v2_part0.csvevent not real
gauge num 491
0.5hrs_2009_v2_part0.csvevent not real
0.5hrs_2013_v2_part0.csvevent not real
0.5hrs_2017_v2_part0.csvevent not real
0.5hrs_2018_v2_part0.csvevent not real
1hrs_2013_v2_part0.csvevent not real
1hrs_2017_v2_part0.csvevent not real
gauge num 492
0.5hrs_2013_v2_part0.csvevent not real
0.5hrs_2017_v2_part0.csvevent not real
0.5hrs_2018_v2_part0.csvevent not real
1hrs_2013_v2_part0.csvevent not real
gauge num 493
gauge num 494
gauge num 495
gauge num 496
0.5hrs_2006_v2_part0.csvevent not real
0.5hrs_2007_v2_par

1hrs_2010_v2_part0.csvevent not real
gauge num 645
1hrs_2009_v2_part0.csvevent not real
gauge num 646
gauge num 647
gauge num 648
gauge num 649
gauge num 650
0.5hrs_2015_v2_part0.csvevent not real
0.5hrs_2018_v2_part0.csvevent not real
24hrs_2010_v2_part1.csvevent not real
gauge num 651
gauge num 652
0.5hrs_2009_v2_part0.csvevent not real
0.5hrs_2013_v2_part0.csvevent not real
0.5hrs_2018_v2_part0.csvevent not real
gauge num 653
gauge num 654
gauge num 655
gauge num 656
0.5hrs_2008_v2_part0.csvevent not real
2hrs_2008_v2_part0.csvevent not real
gauge num 657
0.5hrs_2010_v2_part0.csvevent not real
gauge num 658
0.5hrs_2008_v2_part0.csvevent not real
gauge num 659
gauge num 660
0.5hrs_2018_v2_part0.csvevent not real
gauge num 661
0.5hrs_2009_v2_part0.csvevent not real
gauge num 662
0.5hrs_2016_v2_part0.csvevent not real
1hrs_2016_v2_part0.csvevent not real
gauge num 663
0.5hrs_2017_v2_part0.csvevent not real
gauge num 664
0.5hrs_2011_v2_part0.csvevent not real
gauge num 665
gauge num 666

gauge num 834
gauge num 835
0.5hrs_2006_v2_part0.csvevent not real
0.5hrs_2008_v2_part0.csvevent not real
0.5hrs_2012_v2_part0.csvevent not real
1hrs_2012_v2_part0.csvevent not real
2hrs_2012_v2_part0.csvevent not real
gauge num 836
gauge num 837
gauge num 838
gauge num 839
0.5hrs_2014_v2_part0.csvevent not real
0.5hrs_2019_v2_part0.csvevent not real
gauge num 840
0.5hrs_2010_v2_part0.csvevent not real
1hrs_2008_v2_part0.csvevent not real
gauge num 841
gauge num 842
gauge num 843
gauge num 844
gauge num 845
0.5hrs_2011_v2_part0.csvevent not real
gauge num 846
0.5hrs_2006_v2_part0.csvevent not real
gauge num 847
0.5hrs_2020_v2_part0.csvevent not real
gauge num 848
gauge num 849
gauge num 850
0.5hrs_2017_v2_part0.csvevent not real
gauge num 851
0.5hrs_2009_v2_part0.csvevent not real
gauge num 852
gauge num 853
0.5hrs_2006_v2_part0.csvevent not real
gauge num 854
gauge num 855
gauge num 856
gauge num 857
0.5hrs_2008_v2_part0.csvevent not real
0.5hrs_2014_v2_part0.csvevent not real
0.5hrs_

gauge num 1086
gauge num 1087
0.5hrs_2007_v2_part0.csvevent not real
0.5hrs_2015_v2_part0.csvevent not real
1hrs_2007_v2_part0.csvevent not real
gauge num 1088
gauge num 1089
gauge num 1090
0.5hrs_2006_v2_part0.csvevent not real
0.5hrs_2010_v2_part0.csvevent not real
gauge num 1091
gauge num 1092
gauge num 1093
gauge num 1094
gauge num 1095
gauge num 1096
gauge num 1097
gauge num 1098
0.5hrs_2017_v2_part0.csvevent not real
gauge num 1099
0.5hrs_2010_v2_part0.csvevent not real
1hrs_2010_v2_part0.csvevent not real
gauge num 1100
gauge num 1101
gauge num 1102
gauge num 1103
gauge num 1104
gauge num 1105
1hrs_2017_v2_part0.csvevent not real
gauge num 1106
gauge num 1107
gauge num 1108
0.5hrs_2014_v2_part0.csvevent not real
24hrs_2006_v2_part1.csvevent not real
gauge num 1109
gauge num 1110
0.5hrs_2016_v2_part0.csvevent not real
1hrs_2016_v2_part0.csvevent not real
gauge num 1111
gauge num 1112
gauge num 1113
gauge num 1114
0.5hrs_2008_v2_part0.csvevent not real
0.5hrs_2009_v2_part0.csveven

gauge num 1290
0.5hrs_2006_v2_part0.csvevent not real
gauge num 1291
gauge num 1292


In [284]:
# # Initialize an empty DataFrame with the desired columns
columns = [
    'gauge_num',  'season', 'precip', 'Volume','Year', 'duration',
    'normalized_rainfall', 'normalized_interpolated_rainfall_12', 'normalized_interpolated_rainfall_15',
    'max_quintile_profile_12', 'max_quintile_profile_15', 'max_quintile_normalised_rain', 'max_quintile_raw_rain']
df = pd.DataFrame(columns=columns)

for nimrod_option in ["NIMROD_2.2km_filtered_100"]:
    for gauge_num in range(0, 1293):
        if gauge_num not in [444, 827, 888]:
            print(f"gauge {gauge_num}")
            files = [f for f in os.listdir(f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_30mins/{nimrod_option}/{gauge_num}/WholeYear") if f.endswith('.csv')]
            files = np.sort(files)

            for file in files:
                fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_30mins/{nimrod_option}/{gauge_num}/WholeYear/{file}"
                if '2080' in fp:
                    continue

                this_event = read_event(gauge_num, fp)
                trimmed_event = remove_leading_and_trailing_zeroes(this_event)
                real_trimmed_event, problem_events = remove_events_with_problems(trimmed_event, verbose=False)

                if real_trimmed_event is not None:
                    precip = real_trimmed_event['precipitation (mm/hr)']
                    normalized_time, normalized_rainfall = create_normalised_event(real_trimmed_event)
                    normalized_interpolated_rainfall_12 = interpolate_and_bin(normalized_time, normalized_rainfall, 12)
                    normalized_interpolated_rainfall_15 = interpolate_and_bin(normalized_time, normalized_rainfall, 15)
                    max_quintile_profile_12 = find_part_with_most_rain_using_cumulative_rainfall(normalized_interpolated_rainfall_12, 5)
                    max_quintile_profile_15 = find_part_with_most_rain_using_cumulative_rainfall(normalized_interpolated_rainfall_15, 5)
                    max_quintile_normalised_rain = find_part_with_most_rain_using_cumulative_rainfall(normalized_rainfall, 5)
                    max_quintile_raw_rain = find_part_with_most_rain(real_trimmed_event['precipitation (mm/hr)'], 5)
                    duration = len(real_trimmed_event) / 2
                    season = get_season(trimmed_event['times'][0])
                else:
                    precip=trimmed_event['precipitation (mm/hr)']
                    normalized_rainfall = None
                    normalized_interpolated_rainfall_12 = None
                    normalized_interpolated_rainfall_15 = None
                    max_quintile_profile_12 = None
                    max_quintile_profile_15 = None
                    max_quintile_normalised_rain = None
                    max_quintile_raw_rain = None
                    duration = None
                    season = None
                # Append the row to the DataFrame
                df = df.append({
                    'gauge_num': gauge_num,
                    'season': season,
                    'precip': precip.values,
                    'Volume': sum(precip),
                    'Year':extract_year(trimmed_event),
                    'duration': duration,
                    'normalized_rainfall': normalized_rainfall,
                    'normalized_interpolated_rainfall_12': normalized_interpolated_rainfall_12,
                    'normalized_interpolated_rainfall_15': normalized_interpolated_rainfall_15,
                    'max_quintile_profile_12': max_quintile_profile_12,
                    'max_quintile_profile_15': max_quintile_profile_15,
                    'max_quintile_normalised_rain': max_quintile_normalised_rain,
                    'max_quintile_raw_rain': max_quintile_raw_rain
                }, ignore_index=True)
df['Loading_profile12'] = df['max_quintile_profile_12'].map(quintile_mapping)
df['Loading_profile15'] = df['max_quintile_profile_15'].map(quintile_mapping)
df['Loading_profile_normalised_rain'] = df['max_quintile_normalised_rain'].map(quintile_mapping)
df['Loading_profile_raw_rain'] = df['max_quintile_raw_rain'].map(quintile_mapping)

with open(f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/NIMROD_30mins/WholeYear/{nimrod_option}_profiles_df.pkl", 'wb') as file:
    pickle.dump(df, file)

gauge 0
gauge 1
gauge 2
gauge 3
gauge 4
gauge 5
gauge 6
gauge 7
gauge 8
gauge 9
gauge 10
gauge 11
gauge 12
gauge 13
gauge 14
gauge 15
gauge 16
gauge 17
gauge 18
gauge 19
gauge 20
gauge 21
gauge 22
gauge 23
gauge 24
gauge 25
gauge 26
gauge 27
gauge 28
gauge 29
gauge 30
gauge 31
gauge 32
gauge 33
gauge 34
gauge 35
gauge 36
gauge 37
gauge 38
gauge 39
gauge 40
gauge 41
gauge 42
gauge 43
gauge 44
gauge 45
gauge 46
gauge 47
gauge 48
gauge 49
gauge 50
gauge 51
gauge 52
gauge 53
gauge 54
gauge 55
gauge 56
gauge 57
gauge 58
gauge 59
gauge 60
gauge 61
gauge 62
gauge 63
gauge 64
gauge 65
gauge 66
gauge 67
gauge 68
gauge 69
gauge 70
gauge 71
gauge 72
gauge 73
gauge 74
gauge 75
gauge 76
gauge 77
gauge 78
gauge 79
gauge 80
gauge 81
gauge 82
gauge 83
gauge 84
gauge 85
gauge 86
gauge 87
gauge 88
gauge 89
gauge 90
gauge 91
gauge 92
gauge 93
gauge 94
gauge 95
gauge 96
gauge 97
gauge 98
gauge 99
gauge 100
gauge 101
gauge 102
gauge 103
gauge 104
gauge 105
gauge 106
gauge 107
gauge 108
gauge 109
gauge 110


gauge 833
gauge 834
gauge 835
gauge 836
gauge 837
gauge 838
gauge 839
gauge 840
gauge 841
gauge 842
gauge 843
gauge 844
gauge 845
gauge 846
gauge 847
gauge 848
gauge 849
gauge 850
gauge 851
gauge 852
gauge 853
gauge 854
gauge 855
gauge 856
gauge 857
gauge 858
gauge 859
gauge 860
gauge 861
gauge 862
gauge 863
gauge 864
gauge 865
gauge 866
gauge 867
gauge 868
gauge 869
gauge 870
gauge 871
gauge 872
gauge 873
gauge 874
gauge 875
gauge 876
gauge 877
gauge 878
gauge 879
gauge 880
gauge 881
gauge 882
gauge 883
gauge 884
gauge 885
gauge 886
gauge 887
gauge 889
gauge 890
gauge 891
gauge 892
gauge 893
gauge 894
gauge 895
gauge 896
gauge 897
gauge 898
gauge 899
gauge 900
gauge 901
gauge 902
gauge 903
gauge 904
gauge 905
gauge 906
gauge 907
gauge 908
gauge 909
gauge 910
gauge 911
gauge 912
gauge 913
gauge 914
gauge 915
gauge 916
gauge 917
gauge 918
gauge 919
gauge 920
gauge 921
gauge 922
gauge 923
gauge 924
gauge 925
gauge 926
gauge 927
gauge 928
gauge 929
gauge 930
gauge 931
gauge 932
gauge 933


# NIMROD 5 mins

In [112]:
for nimrod_option in ["NIMROD_1km_filtered_300"]:

    # New list to store just the files that are 'good' e.g. 
    dict_of_things_im_storing ={}
    dict_of_things_im_storing['good_raw_events'] = []
    dict_of_things_im_storing['good_raw_events_seasons'] = []
    dict_of_things_im_storing['good_trimmed_events'] = []
    dict_of_things_im_storing['good_trimmed_events_durations'] = []
    dict_of_things_im_storing['good_normalised_events'] = []
    dict_of_things_im_storing['good_normalised_interpolated_events'] = []
    dict_of_things_im_storing['good_normalised_interpolated_events_max_quintiles'] = []

    for gauge_num in range(0,1262):
        if gauge_num not in [443, 444, 827, 888]:
            print(gauge_num)

            # Create a list of all the event CSVs
            files = [f for f in os.listdir(f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_5mins/{nimrod_option}/{gauge_num}/WholeYear") if f.endswith('.csv')]
            files = np.sort(files)

            # Go through each event
            for file in files:
                # Construct the path to the file
                fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_5mins/{nimrod_option}/{gauge_num}/WholeYear/{file}"
                # Read in event
                this_event = read_event(gauge_num, fp)
                # Trim leading and trailing zeroes
                trimmed_event = remove_leading_and_trailing_zeroes(this_event)

                # Check trimmed event for 'problems', if there are none, return a dataframe, if there are, return None
                trimmed_event = remove_events_with_problems(trimmed_event, verbose=True)

                # If we still have a dataframe, then add it to list of good, unprocessed events
                if trimmed_event is not None:

                    # Create normalised version of event
                    normalized_time, normalized_rainfall = create_normalised_event(trimmed_event)
                    # Create interpolated, normalised version of event (with 12 data points)
                    normalized_interpolated_rainfall = interpolate_and_bin(normalized_time, normalized_rainfall)
                    # Find the fifth OF THE INTERPOLATED, NORMALISED EVENT with the maximum amount of rainfall
                    max_quintile = find_fifth_with_most_rain(normalized_interpolated_rainfall)

                    # Add to list of events with no problems
                    dict_of_things_im_storing['good_raw_events'].append(this_event)
                    dict_of_things_im_storing['good_raw_events_seasons'].append(get_season(this_event['times'][0]))
                    dict_of_things_im_storing['good_trimmed_events'].append(trimmed_event)
                    dict_of_things_im_storing['good_trimmed_events_durations'].append(len(trimmed_event)/12)
                    dict_of_things_im_storing['good_normalised_events'].append(normalized_rainfall)
                    dict_of_things_im_storing['good_normalised_interpolated_events'].append(normalized_interpolated_rainfall)
                    dict_of_things_im_storing['good_normalised_interpolated_events_max_quintiles'].append(max_quintile)

    for file_name,stored_list in dict_of_things_im_storing.items():
        print(file_name)
        cache_filepath = f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/NIMROD_5mins/WholeYear/{nimrod_option}_{file_name}.pkl"
        with open(cache_filepath, 'wb') as f:
            pickle.dump(stored_list, f)                    

0
1
2
3
4
5
Doesn't contain more than 1 value which isn't 0
6
7
8
Doesn't contain more than 1 value which isn't 0
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
Doesn't contain more than 1 value which isn't 0
31
32
33
34
35
36
Doesn't contain more than 1 value which isn't 0
Doesn't contain more than 1 value which isn't 0
37
38
39
40
41
42
Doesn't contain more than 1 value which isn't 0
43
Doesn't contain more than 1 value which isn't 0
44
Too short to be an event
45
Too short to be an event
Too short to be an event
Too short to be an event
Too short to be an event
Too short to be an event
Too short to be an event
Too short to be an event
46
47
48
49
50
51
Doesn't contain more than 1 value which isn't 0
Too short to be an event
52
53
54
Doesn't contain more than 1 value which isn't 0
55
56
57
58
59
60
61
62
63
64
65
66
Doesn't contain more than 1 value which isn't 0
67
68
69
Too short to be an event
Too short to be an event
Too short to be an event
Too short to be an e

1224
1225
1226
1227
1228
1229
1230
1231
1232
1233
1234
1235
1236
1237
1238
1239
1240
1241
1242
1243
1244
1245
1246
1247
1248
1249
1250
1251
1252
1253
1254
1255
1256
1257
1258
1259
1260
1261
good_raw_events
good_raw_events_seasons
good_trimmed_events
good_trimmed_events_durations
good_normalised_events
good_normalised_interpolated_events
good_normalised_interpolated_events_max_quintiles


# UKCP18
### Make and pickle profiles

In [273]:
# for em in ["bc005"]:

#     # New list to store just the files that are 'good' e.g. 
#     dict_of_things_im_storing ={}
#     dict_of_things_im_storing['good_raw_events'] = []
#     dict_of_things_im_storing['good_raw_events_seasons'] = []
#     dict_of_things_im_storing['good_trimmed_events'] = []
#     dict_of_things_im_storing['good_trimmed_events_durations'] = []
#     dict_of_things_im_storing['good_normalised_events'] = []
#     dict_of_things_im_storing['good_normalised_interpolated_events'] = []
#     dict_of_things_im_storing['good_normalised_interpolated_events_max_quintiles'] = []

#     for gauge_num in range(0,1294):
#         if gauge_num not in [444, 827, 888]:
#             print(f"gauge {gauge_num}")
#             files = [f for f in os.listdir(f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/UKCP18_30mins/{em}/{gauge_num}/WholeYear/") if f.endswith('.csv')]
#             files = np.sort(files)

#             # Go through each event
#             for file in files:
#                 # Construct the path to the file
#                 fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/UKCP18_30mins/{em}/{gauge_num}/WholeYear/{file}"
#                 if '2080' in fp:
#                     pass
#                     # print(f"{fp} 2080 so not using")
#                 else:
#                     # print(f"{fp}not 2080 so using")
#                     # Read in event
#                     this_event = read_event(gauge_num, fp)
#                     # Trim leading and trailing zeroes
#                     trimmed_event = remove_leading_and_trailing_zeroes(this_event)
                    
#                     # Check trimmed event for 'problems', if there are none, return a dataframe, if there are, return None
#                     real_trimmed_event, problem_events = remove_events_with_problems(trimmed_event, verbose=False)

#                     # If we still have a dataframe, then add it to list of good, unprocessed events
#                     if real_trimmed_event is not None:

#                         # Create normalised version of event
#                         normalized_time, normalized_rainfall = create_normalised_event(real_trimmed_event)
#                         print(len(normalized_rainfall))
#                         # Create interpolated, normalised version of event (with 12 data points)
#                         normalized_interpolated_rainfall_12 = interpolate_and_bin(normalized_time, normalized_rainfall, 12)
#                         normalized_interpolated_rainfall_15 = interpolate_and_bin(normalized_time, normalized_rainfall, 15)
                        
#                         # Find the fifth OF THE INTERPOLATED, NORMALISED EVENT with the maximum amount of rainfall
#                         max_quintile_profile_12 = find_part_with_most_rain(normalized_interpolated_rainfall_12, 5)
#                         max_quintile_profile_15 = find_part_with_most_rain(normalized_interpolated_rainfall_15, 5)
#                         max_quintile_raw_rain = find_part_with_most_rain(normalized_rainfall, 5)
                        
#                         duration = len(real_trimmed_event)/2
#                         season = get_season(trimmed_event['times'][0])
                                            
#                     else:
#                         print(f"{file}event not real")
#                         normalized_rainfall = None
#                         normalized_interpolated_rainfall_12 = None
#                         normalized_interpolated_rainfall_15 = None
#                         max_quintile_profile_12 = None
#                         max_quintile_profile_15 = None
#                         max_quintile_raw_rain = None
#                         duration = None
#                         season = None

#                     # Add to list of events with no problems
#                     dict_of_things_im_storing['good_raw_events'].append(this_event)
#                     dict_of_things_im_storing['good_raw_events_seasons'].append(season)
#                     dict_of_things_im_storing['good_trimmed_events'].append(real_trimmed_event)
#                     dict_of_things_im_storing['good_trimmed_events_durations'].append(duration)
#                     dict_of_things_im_storing['good_normalised_events'].append(normalized_rainfall)
#                     dict_of_things_im_storing['good_normalised_interpolated_events'].append(normalized_interpolated_rainfall)
#                     dict_of_things_im_storing['good_normalised_interpolated_events_max_quintiles'].append(max_quintile)

#         print(len(dict_of_things_im_storing['good_normalised_interpolated_events_max_quintiles']))
        
#     for file_name,stored_list in dict_of_things_im_storing.items():
#         print(file_name)
#         cache_filepath = f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/UKCP18_30mins/{em}/new_{file_name}.pkl"
#         with open(cache_filepath, 'wb') as f:
#             pickle.dump(stored_list, f)                        

In [279]:
def interpolate_and_bin(normalized_time, normalized_rainfall, bin_number):
    """
    Interpolate missing data points and bin the dimensionless profile into 12 segments.
    
    Parameters:
        normalized_time (np.array): Normalized time array.
        normalized_rainfall (np.array): Normalized cumulative rainfall array.
    
    Returns:
        np.array: Binned and interpolated rainfall profile.
    """
    # Define target points for 12 bins
    target_points = np.linspace(0, 1, bin_number+1)
    
    # Create interpolation function based on existing data points
    interpolation_func = interp1d(normalized_time, normalized_rainfall, kind='linear', fill_value="extrapolate")
    
    # Interpolate values at target points
    interpolated_values = interpolation_func(target_points)
    
    return interpolated_values

In [280]:
# # Initialize an empty DataFrame with the desired columns
columns = [
    'gauge_num',  'season', 'precip', 'Volume','Year', 'duration',
    'normalized_rainfall', 'normalized_interpolated_rainfall_12', 'normalized_interpolated_rainfall_15',
    'max_quintile_profile_12', 'max_quintile_profile_15', 'max_quintile_normalised_rain', 'max_quintile_raw_rain']
df = pd.DataFrame(columns=columns)

for em in ["bb198"]:
    for gauge_num in range(0, 1293):
        if gauge_num not in [444, 827, 888]:
            print(f"gauge {gauge_num}")
            files = [f for f in os.listdir(f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/UKCP18_30mins/{em}/{gauge_num}/WholeYear/") if f.endswith('.csv')]
            files = np.sort(files)

            for file in files:
                fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/UKCP18_30mins/{em}/{gauge_num}/WholeYear/{file}"
                if '2080' in fp:
                    continue

                this_event = read_event(gauge_num, fp)
                trimmed_event = remove_leading_and_trailing_zeroes(this_event)
                real_trimmed_event, problem_events = remove_events_with_problems(trimmed_event, verbose=False)

                if real_trimmed_event is not None:
                    precip = real_trimmed_event['precipitation (mm/hr)']
                    normalized_time, normalized_rainfall = create_normalised_event(real_trimmed_event)
                    normalized_interpolated_rainfall_12 = interpolate_and_bin(normalized_time, normalized_rainfall, 12)
                    normalized_interpolated_rainfall_15 = interpolate_and_bin(normalized_time, normalized_rainfall, 15)
                    max_quintile_profile_12 = find_part_with_most_rain_using_cumulative_rainfall(normalized_interpolated_rainfall_12, 5)
                    max_quintile_profile_15 = find_part_with_most_rain_using_cumulative_rainfall(normalized_interpolated_rainfall_15, 5)
                    max_quintile_normalised_rain = find_part_with_most_rain_using_cumulative_rainfall(normalized_rainfall, 5)
                    max_quintile_raw_rain = find_part_with_most_rain(real_trimmed_event['precipitation (mm/hr)'], 5)
                    duration = len(real_trimmed_event) / 2
                    season = get_season(trimmed_event['times'][0])
                else:
                    precip=trimmed_event['precipitation (mm/hr)']
                    normalized_rainfall = None
                    normalized_interpolated_rainfall_12 = None
                    normalized_interpolated_rainfall_15 = None
                    max_quintile_profile_12 = None
                    max_quintile_profile_15 = None
                    max_quintile_normalised_rain = None
                    max_quintile_raw_rain = None
                    duration = None
                    season = None
                # Append the row to the DataFrame
                df = df.append({
                    'gauge_num': gauge_num,
                    'season': season,
                    'precip': precip.values,
                    'Volume': sum(precip),
                    'Year':extract_year(trimmed_event),
                    'duration': duration,
                    'normalized_rainfall': normalized_rainfall,
                    'normalized_interpolated_rainfall_12': normalized_interpolated_rainfall_12,
                    'normalized_interpolated_rainfall_15': normalized_interpolated_rainfall_15,
                    'max_quintile_profile_12': max_quintile_profile_12,
                    'max_quintile_profile_15': max_quintile_profile_15,
                    'max_quintile_normalised_rain': max_quintile_normalised_rain,
                    'max_quintile_raw_rain': max_quintile_raw_rain
                }, ignore_index=True)
df['Loading_profile12'] = df['max_quintile_profile_12'].map(quintile_mapping)
df['Loading_profile15'] = df['max_quintile_profile_15'].map(quintile_mapping)
df['Loading_profile_normalised_rain'] = df['max_quintile_normalised_rain'].map(quintile_mapping)
df['Loading_profile_raw_rain'] = df['max_quintile_raw_rain'].map(quintile_mapping)

with open(f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/UKCP18_30mins/{em}/profiles_df.pkl", 'wb') as file:
    pickle.dump(df, file)

gauge 0
gauge 1
gauge 2
gauge 3
gauge 4
gauge 5
gauge 6
gauge 7
gauge 8
gauge 9
gauge 10
Some dates were invalid and have been coerced to NaT:
   Unnamed: 0  precipitation (mm/hr)                times  precipitation (mm)  \
0        2796               7.537408  2074-02-29 06:15:00            3.768704   
1        2797               9.424045  2074-02-29 06:45:00            4.712022   
2        2798               8.022667  2074-02-29 07:15:00            4.011334   
3        2799               0.934461  2074-02-29 07:45:00            0.467230   
4        2800               0.270409  2074-02-29 08:15:00            0.135204   
5        2801               0.806858  2074-02-29 08:45:00            0.403429   

   is_dry  Rolling_Sum  consecutive_dry timestamp  time_since_last_minutes  
0   False     3.845104              0.0       NaT                      0.0  
1   False     8.480726              0.0       NaT                      0.0  
2   False     8.723356              NaN       NaT         

gauge 29
Some dates were invalid and have been coerced to NaT:
   Unnamed: 0  precipitation (mm/hr)                times  precipitation (mm)  \
0        2877               1.321049  2069-02-30 22:45:00            0.660525   
1        2878               1.354208  2069-02-30 23:15:00            0.677104   
2        2879               2.156420  2069-02-30 23:45:00            1.078210   

   is_dry  Rolling_Sum  consecutive_dry timestamp  time_since_last_minutes  
0   False     6.202075              NaN       NaT                      0.0  
1   False     6.838645              NaN       NaT                      0.0  
2   False     7.897704              NaN       NaT                      0.0  
gauge 30
gauge 31
gauge 32
gauge 33
Some dates were invalid and have been coerced to NaT:
    Unnamed: 0  precipitation (mm/hr)                times  \
0         2793           1.367123e+00  2074-02-29 04:45:00   
1         2794           1.496621e-01  2074-02-29 05:15:00   
2         2795           1.0

gauge 38
gauge 39
gauge 40
gauge 41
gauge 42
gauge 43
gauge 44
gauge 45
gauge 46
gauge 47
gauge 48
gauge 49
gauge 50
gauge 51
gauge 52
gauge 53
gauge 54
gauge 55
gauge 56
gauge 57
Some dates were invalid and have been coerced to NaT:
    Unnamed: 0  precipitation (mm/hr)                times  \
0         2835               0.001172  2065-02-30 01:45:00   
1         2836               1.206281  2065-02-30 02:15:00   
2         2837               2.137789  2065-02-30 02:45:00   
3         2838               2.464675  2065-02-30 03:15:00   
4         2839               1.975674  2065-02-30 03:45:00   
5         2840               2.061272  2065-02-30 04:15:00   
6         2841               2.062744  2065-02-30 04:45:00   
7         2842               2.027637  2065-02-30 05:15:00   
8         2843               2.010471  2065-02-30 05:45:00   
9         2844               2.025466  2065-02-30 06:15:00   
10        2845               2.096147  2065-02-30 06:45:00   
11        2846        

gauge 59
Some dates were invalid and have been coerced to NaT:
    Unnamed: 0  precipitation (mm/hr)                times  \
0         2837               0.021962  2065-02-30 02:45:00   
1         2838               1.545390  2065-02-30 03:15:00   
2         2839               2.211054  2065-02-30 03:45:00   
3         2840               2.160278  2065-02-30 04:15:00   
4         2841               2.083468  2065-02-30 04:45:00   
5         2842               2.074890  2065-02-30 05:15:00   
6         2843               1.995335  2065-02-30 05:45:00   
7         2844               2.131113  2065-02-30 06:15:00   
8         2845               2.112516  2065-02-30 06:45:00   
9         2846               2.504502  2065-02-30 07:15:00   
10        2847               2.606928  2065-02-30 07:45:00   
11        2848               2.445049  2065-02-30 08:15:00   
12        2849               2.729105  2065-02-30 08:45:00   
13        2850               2.704219  2065-02-30 09:15:00   
14     

Some dates were invalid and have been coerced to NaT:
    Unnamed: 0  precipitation (mm/hr)                times  \
0         2796               0.183846  2061-02-29 06:15:00   
1         2797               0.705110  2061-02-29 06:45:00   
2         2798               1.111584  2061-02-29 07:15:00   
3         2799               1.510752  2061-02-29 07:45:00   
4         2800               1.649484  2061-02-29 08:15:00   
5         2801               1.779964  2061-02-29 08:45:00   
6         2802               1.842606  2061-02-29 09:15:00   
7         2803               1.881930  2061-02-29 09:45:00   
8         2804               3.504502  2061-02-29 10:15:00   
9         2805               4.079005  2061-02-29 10:45:00   
10        2806               3.800992  2061-02-29 11:15:00   
11        2807               5.893722  2061-02-29 11:45:00   
12        2808               4.392496  2061-02-29 12:15:00   
13        2809               4.461803  2061-02-29 12:45:00   
14        2810  

...

gauge 1237
gauge 1238
gauge 1239
Some dates were invalid and have been coerced to NaT:
    Unnamed: 0  precipitation (mm/hr)                times  \
0         2833               0.000009  2072-02-30 00:45:00   
1         2834               0.000018  2072-02-30 01:15:00   
2         2835               0.000022  2072-02-30 01:45:00   
3         2836               0.675795  2072-02-30 02:15:00   
4         2837               1.072376  2072-02-30 02:45:00   
5         2838               1.439549  2072-02-30 03:15:00   
6         2839               1.379138  2072-02-30 03:45:00   
7         2840               0.869642  2072-02-30 04:15:00   
8         2841               1.113992  2072-02-30 04:45:00   
9         2842               0.810780  2072-02-30 05:15:00   
10        2843               1.168907  2072-02-30 05:45:00   
11        2844               1.206370  2072-02-30 06:15:00   
12        2845               1.386446  2072-02-30 06:45:00   
13        2846               1.573291  2072-0

gauge 1248
gauge 1249
gauge 1250
gauge 1251
gauge 1252
gauge 1253
gauge 1254
gauge 1255
gauge 1256
gauge 1257
Some dates were invalid and have been coerced to NaT:
    Unnamed: 0  precipitation (mm/hr)                times  \
23        2784               3.219521  2061-02-29 00:15:00   
24        2785               3.550124  2061-02-29 00:45:00   
25        2786               3.551927  2061-02-29 01:15:00   
26        2787               3.431825  2061-02-29 01:45:00   
27        2788               3.746678  2061-02-29 02:15:00   
28        2789               3.585306  2061-02-29 02:45:00   
29        2790               3.504385  2061-02-29 03:15:00   
30        2791               3.352582  2061-02-29 03:45:00   
31        2792               3.288509  2061-02-29 04:15:00   
32        2793               3.120410  2061-02-29 04:45:00   
33        2794               3.176660  2061-02-29 05:15:00   
34        2795               3.071707  2061-02-29 05:45:00   
35        2796               3

gauge 1259
gauge 1260
gauge 1261
gauge 1262
gauge 1263
gauge 1264
Some dates were invalid and have been coerced to NaT:
    Unnamed: 0  precipitation (mm/hr)                times  \
42        2784               0.443946  2077-02-29 00:15:00   
43        2785               0.112119  2077-02-29 00:45:00   
44        2786               1.263992  2077-02-29 01:15:00   

    precipitation (mm)  is_dry  Rolling_Sum  consecutive_dry timestamp  \
42            0.221973   False    45.602318              NaN       NaT   
43            0.056059   False    43.444682              NaN       NaT   
44            0.631996   False    41.768253              NaN       NaT   

    time_since_last_minutes  
42                      0.0  
43                      0.0  
44                      0.0  
Some dates were invalid and have been coerced to NaT:
    Unnamed: 0  precipitation (mm/hr)                times  \
45        2784               0.443946  2077-02-29 00:15:00   
46        2785               0.11211

gauge 1268
Some dates were invalid and have been coerced to NaT:
    Unnamed: 0  precipitation (mm/hr)                times  \
0         2837               2.666303  2072-02-30 02:45:00   
1         2838               0.011316  2072-02-30 03:15:00   
2         2839               2.890544  2072-02-30 03:45:00   
3         2840               0.015673  2072-02-30 04:15:00   
4         2841               3.844763  2072-02-30 04:45:00   
5         2842               0.058193  2072-02-30 05:15:00   
6         2843               5.706568  2072-02-30 05:45:00   
7         2844               0.180226  2072-02-30 06:15:00   
8         2845               5.313220  2072-02-30 06:45:00   
9         2846               0.305479  2072-02-30 07:15:00   
10        2847               5.324649  2072-02-30 07:45:00   
11        2848               0.573109  2072-02-30 08:15:00   
12        2849               5.506865  2072-02-30 08:45:00   
13        2850               0.217153  2072-02-30 09:15:00   
14   

gauge 1269
gauge 1270
Some dates were invalid and have been coerced to NaT:
    Unnamed: 0  precipitation (mm/hr)                times  \
0         2790           5.583630e-01  2074-02-29 03:15:00   
1         2791           5.137740e-01  2074-02-29 03:45:00   
2         2792           1.059089e+00  2074-02-29 04:15:00   
3         2793           5.254829e+00  2074-02-29 04:45:00   
4         2794           1.067378e+01  2074-02-29 05:15:00   
5         2795           1.113441e+01  2074-02-29 05:45:00   
6         2796           1.956838e+00  2074-02-29 06:15:00   
7         2797           8.443192e-01  2074-02-29 06:45:00   
8         2798           3.156364e+00  2074-02-29 07:15:00   
9         2799           9.241077e+00  2074-02-29 07:45:00   
10        2800           4.770729e-02  2074-02-29 08:15:00   
11        2801           1.220363e-02  2074-02-29 08:45:00   
12        2802           1.194025e-07  2074-02-29 09:15:00   
13        2803           1.190455e-06  2074-02-29 09:45: